<a href="https://colab.research.google.com/github/joel-d-IAya/iaya-v2-staging/blob/main/Script_d'Injection_Directus.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import requests
import json
import time

# --- CONFIGURATION IAYA ---
DIRECTUS_URL = "https://cms.iaya.cloud"
API_TOKEN = "MkFqoSUe6Xxxb8nPumDjPiKBTxRLvZqT" # À récupérer dans ton Vault
HEADERS = {
    "Authorization": f"Bearer {API_TOKEN}",
    "Content-Type": "application/json"
}

# On ne traite que tes deux fichiers consolidés
FILES_TO_PROCESS = [
    {"file": "Contents services section - services.csv", "base": "services", "trans": "services_translations", "ref": "services_id"},
    {"file": "Contents services section - sub_services.csv", "base": "sub_services", "trans": "sub_services_translations", "ref": "sub_services_id"}
]

def clean_val(val):
    if pd.isna(val) or val == "" or str(val).lower() == "nan": return None
    if str(val).upper() == "TRUE": return True
    if str(val).upper() == "FALSE": return False
    if isinstance(val, str) and val.startswith("["):
        try: return json.loads(val.replace("'", '"'))
        except: return val
    return val

def get_translation_id(trans_collection, parent_ref, parent_id, lang_code):
    """Trouve l'ID de la ligne de traduction existante."""
    url = f"{DIRECTUS_URL}/items/{trans_collection}"
    params = {
        "filter": json.dumps({
            "_and": [
                {parent_ref: {"_eq": parent_id}},
                {"languages_code": {"_eq": lang_code}}
            ]
        }),
        "fields": "id"
    }
    try:
        r = requests.get(url, headers=HEADERS, params=params)
        data = r.json().get('data', [])
        return data[0]['id'] if data else None
    except:
        return None

def run_update():
    for task in FILES_TO_PROCESS:
        print(f"\n📂 Analyse du fichier : {task['file']}")
        try:
            df = pd.read_csv(task['file']).fillna("")

            for _, row in df.iterrows():
                item_id = int(row['id'])

                # 1. Préparation des données de base (Champs non-traduits)
                base_payload = {}
                for col in df.columns:
                    if col != 'id' and "_fr-" not in col and "_es-" not in col and "_en-" not in col:
                        base_payload[col] = clean_val(row[col])

                # UPDATE TABLE PRINCIPALE
                requests.patch(f"{DIRECTUS_URL}/items/{task['base']}/{item_id}", headers=HEADERS, json=base_payload)
                print(f"✅ Base {task['base']} ID {item_id} mise à jour.")

                # 2. Mise à jour des TRADUCTIONS (fr, es, en)
                languages = {"fr-FR": "fr-FR", "es-ES": "es-ES", "en-US": "en-US"}
                for suffix, lang_code in languages.items():
                    trans_payload = {}
                    # On cherche les champs title, summary, full_content pour cette langue
                    has_data = False
                    for field in ["title", "summary", "full_content"]:
                        col_name = f"{field}_{suffix}"
                        if col_name in row:
                            trans_payload[field] = clean_val(row[col_name])
                            has_data = True

                    if has_data:
                        t_id = get_translation_id(task['trans'], task['ref'], item_id, lang_code)
                        if t_id:
                            requests.patch(f"{DIRECTUS_URL}/items/{task['trans']}/{t_id}", headers=HEADERS, json=trans_payload)
                            print(f"   ∟ Traduction {lang_code} mise à jour (ID: {t_id}).")
                        else:
                            # Si la ligne de traduction n'existe pas, on la crée
                            trans_payload[task['ref']] = item_id
                            trans_payload["languages_code"] = lang_code
                            requests.post(f"{DIRECTUS_URL}/items/{task['trans']}", headers=HEADERS, json=trans_payload)
                            print(f"   ∟ Traduction {lang_code} créée.")

                time.sleep(0.1)
        except Exception as e:
            print(f"❌ Erreur sur {task['file']} : {e}")

if __name__ == "__main__":
    run_update()


📂 Analyse du fichier : Contents services section - services.csv
✅ Base services ID 17 mise à jour.
   ∟ Traduction fr-FR mise à jour (ID: 25).
   ∟ Traduction es-ES mise à jour (ID: 26).
   ∟ Traduction en-US mise à jour (ID: 27).
✅ Base services ID 18 mise à jour.
   ∟ Traduction fr-FR mise à jour (ID: 28).
   ∟ Traduction es-ES mise à jour (ID: 29).
   ∟ Traduction en-US mise à jour (ID: 30).
✅ Base services ID 19 mise à jour.
   ∟ Traduction fr-FR mise à jour (ID: 31).
   ∟ Traduction es-ES mise à jour (ID: 32).
   ∟ Traduction en-US mise à jour (ID: 33).
✅ Base services ID 20 mise à jour.
   ∟ Traduction fr-FR mise à jour (ID: 34).
   ∟ Traduction es-ES mise à jour (ID: 35).
   ∟ Traduction en-US mise à jour (ID: 36).
✅ Base services ID 21 mise à jour.
   ∟ Traduction fr-FR mise à jour (ID: 37).
   ∟ Traduction es-ES mise à jour (ID: 38).
   ∟ Traduction en-US mise à jour (ID: 39).

📂 Analyse du fichier : Contents services section - sub_services.csv
✅ Base sub_services ID 26 mise 